# Video Downsample → Scene Detection → Keyframe Extraction

This notebook combines video downsampling (1920x1080 → 640x360), scene detection, and keyframe extraction. (For dataset batch 2)

In [ ]:
# Install required packages
!pip install opencv-python scenedetect pandas tqdm ffmpeg-python -q

In [1]:
import os
import cv2
import pandas as pd
import glob
import subprocess
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from scenedetect import detect, AdaptiveDetector
from cfg import DOWNLOAD_DIR, SCENE_DIR, KEYFRAME_DIR

In [2]:
# Configuration
TARGET_WIDTH = 640
TARGET_HEIGHT = 360
DOWNSAMPLED_DIR = os.path.join(os.path.dirname(DOWNLOAD_DIR), "downsampled_videos")
MAX_WORKERS = 4

# Create directories
os.makedirs(DOWNSAMPLED_DIR, exist_ok=True)
os.makedirs(SCENE_DIR, exist_ok=True)
os.makedirs(KEYFRAME_DIR, exist_ok=True)

print(f"Source videos: {DOWNLOAD_DIR}")
print(f"Downsampled videos: {DOWNSAMPLED_DIR}")
print(f"Scene data: {SCENE_DIR}")
print(f"Keyframes: {KEYFRAME_DIR}")
print(f"Target resolution: {TARGET_WIDTH}x{TARGET_HEIGHT}")

Source videos: Videos_K02
Downsampled videos: downsampled_videos
Scene data: scene
Keyframes: keyframes_K02
Target resolution: 640x360


In [3]:
# Find all video files
video_list = glob.glob(os.path.join(DOWNLOAD_DIR, '*/*.mp4'))
print(f"Found {len(video_list)} videos to process")

# Display first few videos
for i, video in enumerate(video_list[:5]):
    print(f"{i+1}. {os.path.basename(video)}")
if len(video_list) > 5:
    print(f"... and {len(video_list) - 5} more")

Found 31 videos to process
1. K02_V001.mp4
2. K02_V002.mp4
3. K02_V003.mp4
4. K02_V004.mp4
5. K02_V005.mp4
... and 26 more


In [10]:
def downsample_video_ffmpeg(input_path, output_path, target_width=640, target_height=360):
    """
    Downsample video using FFmpeg with OpenCV-compatible output
    """
    try:
        # Create output directory
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        
        # FFmpeg command optimized for OpenCV compatibility
        cmd = [
            'ffmpeg', '-y',              # Overwrite output files
            '-i', input_path,
            '-vf', f'scale={target_width}:{target_height}',
            '-c:v', 'libx264',          # H.264 codec
            '-preset', 'fast',         # Encoding speed/quality balance  
            '-crf', '22',               # Quality (lower = better)
            output_path
        ]
        
        # Run FFmpeg
        result = subprocess.run(cmd, capture_output=True, text=True, check=True)
        return True, None
        
    except subprocess.CalledProcessError as e:
        error_msg = e.stderr.strip() if e.stderr else str(e)
        return False, f"FFmpeg error: {error_msg}"
    except Exception as e:
        return False, f"Error: {str(e)}"

In [11]:
def process_single_video(video_path):
    """
    Complete pipeline for a single video:
    1. Downsample video with FFmpeg
    2. Detect scenes
    3. Extract keyframes with OpenCV
    """
    base_name = os.path.splitext(os.path.basename(video_path))[0]
    batch = base_name.split("_")[0]
    
    # Paths
    downsampled_path = os.path.join(DOWNSAMPLED_DIR, batch, f"{base_name}.mp4")
    scene_csv_path = os.path.join(SCENE_DIR, batch, f"{base_name}.csv")
    keyframe_dir = os.path.join(KEYFRAME_DIR, batch, base_name)
    
    results = {
        'video': base_name,
        'downsample_success': False,
        'scene_detection_success': False,
        'keyframe_extraction_success': False,
        'errors': []
    }
    
    try:
        # Step 1: Downsample video with FFmpeg (skip if already exists)
        if os.path.exists(downsampled_path):
            results['downsample_success'] = True
        else:
            success, error = downsample_video_ffmpeg(video_path, downsampled_path, TARGET_WIDTH, TARGET_HEIGHT)
            if success:
                results['downsample_success'] = True
            else:
                results['errors'].append(f"Downsampling failed: {error}")
                return results
        
        # Step 2: Scene detection (skip if already exists)
        if os.path.exists(scene_csv_path):
            results['scene_detection_success'] = True
            scene_df = pd.read_csv(scene_csv_path)
        else:
            scene_list = detect(downsampled_path, AdaptiveDetector())
            
            if scene_list:
                scene_df = pd.DataFrame([
                    {
                        "start_frame": start.get_frames(),
                        "start_time": round(start.get_seconds(), 2),
                        "end_frame": end.get_frames(),
                        "end_time": round(end.get_seconds(), 2),
                        "median_frame": (start.get_frames() + end.get_frames()) // 2,
                        "median_time": round((start.get_seconds() + end.get_seconds()) / 2, 2),
                    }
                    for start, end in scene_list
                ])
                
                os.makedirs(os.path.dirname(scene_csv_path), exist_ok=True)
                scene_df.to_csv(scene_csv_path, index=False)
                results['scene_detection_success'] = True
            else:
                results['errors'].append("No scenes detected")
                return results
        
        # Step 3: Extract keyframes with OpenCV
        os.makedirs(keyframe_dir, exist_ok=True)
        
        cap = cv2.VideoCapture(downsampled_path)
        if not cap.isOpened():
            results['errors'].append("Could not open downsampled video for keyframe extraction")
            return results
        
        extracted_count = 0
        for _, row in scene_df.iterrows():
            median_frame = int(row['median_frame'])
            keyframe_path = os.path.join(keyframe_dir, f"{base_name}_{median_frame}.jpg")
            
            # Skip if keyframe already exists
            if os.path.exists(keyframe_path):
                extracted_count += 1
                continue
            
            cap.set(cv2.CAP_PROP_POS_FRAMES, median_frame)
            ret, frame = cap.read()
            if ret:
                cv2.imwrite(keyframe_path, frame)
                extracted_count += 1
        
        cap.release()
        
        if extracted_count > 0:
            results['keyframe_extraction_success'] = True
            results['keyframes_extracted'] = extracted_count
        else:
            results['errors'].append("No keyframes extracted")
        
    except Exception as e:
        results['errors'].append(f"Unexpected error: {str(e)}")
    
    return results

In [ ]:
# Process all videos in parallel
print(f"Processing {len(video_list)} videos with {MAX_WORKERS} workers...\n")

success_count = 0
error_count = 0
all_results = []

with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    # Submit all jobs
    futures = {executor.submit(process_single_video, video): video for video in video_list}
    
    # Process results as they complete
    for future in tqdm(as_completed(futures), total=len(futures), desc="Processing videos"):
        video_path = futures[future]
        try:
            result = future.result()
            all_results.append(result)
            
            if (result['downsample_success'] and 
                result['scene_detection_success'] and 
                result['keyframe_extraction_success']):
                success_count += 1
                print(f"✅ {result['video']}: {result.get('keyframes_extracted', 0)} keyframes")
            else:
                error_count += 1
                print(f"❌ {result['video']}: {', '.join(result['errors'])}")
                
        except Exception as e:
            error_count += 1
            print(f"❌ {os.path.basename(video_path)}: Unexpected error: {str(e)}")

print(f"\n📊 Processing Summary:")
print(f"✅ Successfully processed: {success_count} videos")
print(f"❌ Failed: {error_count} videos")
print(f"📁 Total videos: {len(video_list)}")

Processing 31 videos with 4 workers...



Processing videos:   0%|          | 0/31 [00:00<?, ?it/s]

In [ ]:
# Generate detailed report
print("\n📋 Detailed Error Report:")
print("=" * 50)

for result in all_results:
    if result['errors']:
        print(f"\n❌ {result['video']}:")
        for error in result['errors']:
            print(f"   - {error}")
        print(f"   Status: Downsample={result['downsample_success']}, "
              f"Scenes={result['scene_detection_success']}, "
              f"Keyframes={result['keyframe_extraction_success']}")

print(f"\n\n📈 Processing Statistics:")
print(f"Downsampling success rate: {sum(1 for r in all_results if r['downsample_success']) / len(all_results) * 100:.1f}%")
print(f"Scene detection success rate: {sum(1 for r in all_results if r['scene_detection_success']) / len(all_results) * 100:.1f}%")
print(f"Keyframe extraction success rate: {sum(1 for r in all_results if r['keyframe_extraction_success']) / len(all_results) * 100:.1f}%")

total_keyframes = sum(r.get('keyframes_extracted', 0) for r in all_results)
print(f"Total keyframes extracted: {total_keyframes}")